# Data Modelisation

In [52]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from gensim.utils import ClippedCorpus
import gensim
import tqdm
from gensim.models import Phrases
from gensim.corpora import Dictionary
from nltk.tokenize import RegexpTokenizer
from gensim.models import LdaModel, CoherenceModel, LdaMulticore
from nlp_module import remove_stopwords
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis 

## 1) Data preparation

In [53]:
data = pd.read_csv("datasets/posts_clean.csv")
data.head(3)

,Id,Title,Body,Tags
0,415192,good way create simple python web service,I use python year I little experience python w...,<python><web-services>
1,415344,log implementation prefer,I implement log class c try decide I curious k...,<debugging><language-agnostic><logging>
2,414981,directly modify list element,I struct struct map public int size public map...,<c#><.net>


In [54]:
# remove < and > around Tags
data["Tags"] = data["Tags"].replace({"<" : " "}, regex=True)
data["Tags"] = data["Tags"].replace({">" : " "}, regex=True)

In [55]:
data["Body"] = data["Body"].str.lower()
data["Body"] = data["Body"].apply(remove_stopwords)

In [56]:
data.head(3)

,Id,Title,Body,Tags
0,415192,good way create simple python web service,use python year little experience python web p...,python web-services
1,415344,log implementation prefer,implement log class c try decide curious know ...,debugging language-agnostic logging
2,414981,directly modify list element,struct struct map public int size public map i...,c# .net


In [57]:
docs = data["Body"].to_list()

In [45]:
def extractDigits(lst):
    return [[el] for el in lst]

In [46]:
docs = extractDigits(docs)

In [58]:
# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [59]:
# Compute bigrams.
# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [63]:
# Remove rare and common tokens.
# Create a dictionary representation of the documents.
dct = Dictionary(docs)
dct.filter_extremes(no_below=20, no_above=0.5)

In [64]:
# Bag-of-words representation of the documents.
corpus = [dct.doc2bow(doc) for doc in docs]

In [65]:
print('Number of unique tokens: %d' % len(dct))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 25556
Number of documents: 164598


## 2) Topic Modelling

In [69]:
# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dct[0]  # This is only to "load" the dictionary.
id2word = dct.id2token

model = LdaModel(
    corpus=corpus,
    id2word = id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [70]:
coherence_model_lda = CoherenceModel(model=model, texts=docs, dictionary=dct, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.5735217952362448


In [17]:
def compute_coherence_values(corpus, dictionary, k, a, b):
    lda_model = LdaMulticore(corpus=corpus,
                        id2word = id2word,
                        chunksize=chunksize,
                        alpha=a,
                        eta=b,
                        iterations=iterations,
                        num_topics=k,
                        passes=passes,
                        eval_every=eval_every)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=dct, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [19]:
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=dct, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

100%|██████████| 540/540 [38:30:52<00:00, 256.76s/it]


In [24]:
pd.DataFrame(model_results).sort_values("Coherence", ascending=False)

,Validation_Set,Topics,Alpha,Beta,Coherence
491,100% Corpus,9,0.61,0.31,0.640743
535,100% Corpus,10,asymmetric,0.01,0.636625
195,75% Corpus,8,0.91,0.01,0.629944
252,75% Corpus,10,0.61,0.61,0.629695
221,75% Corpus,9,0.61,0.31,0.626676
...,...,...,...,...,...
15,75% Corpus,2,0.91,0.01,0.440731
290,100% Corpus,2,symmetric,0.01,0.439272
280,100% Corpus,2,0.61,0.01,0.434665
279,100% Corpus,2,0.31,symmetric,0.434100


In [79]:
lda_model = LdaMulticore(corpus=corpus,
                        id2word=id2word,
                        num_topics=9, 
                        random_state=100,
                        chunksize=1000,
                        passes=10,
                        alpha=0.61,
                        eta=0.31)

In [80]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=dct, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.606131361174391


In [81]:
# Visualize the topics
pyLDAvis.enable_notebook()
lda_vis = gensimvis.prepare(lda_model, corpus, dct)
lda_vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.001458 -0.105420       1        1  17.366762
6     -0.146931 -0.108036       2        1  14.679548
4     -0.119965 -0.105013       3        1  13.039700
3      0.026130  0.020896       4        1  11.178078
2     -0.074665 -0.050772       5        1  10.628857
7      0.137524 -0.128516       6        1   9.316631
5     -0.138801  0.229864       7        1   9.122911
1      0.339632  0.060701       8        1   7.387085
8     -0.021466  0.186297       9        1   7.280428, topic_info=         Term          Freq          Total Category  logprob  loglift
93       java  66657.000000   66657.000000  Default  30.0000  30.0000
2078  android  58229.000000   58229.000000  Default  29.0000  29.0000
736       div  51746.000000   51746.000000  Default  28.0000  28.0000
310      file  89758.000000   89758.000000  Default  27.0000  27.0000
75        int  80445.000000   80445.000000  Default  26.0000  26.0000
...       ...           ...            ...      ...      ...      ...
40      class   6872.633449   83982.943485   Topic9  -5.1381   0.1169
0         add   5812.792815   47271.006080   Topic9  -5.3056   0.5241
68        try   6070.476782   85270.664376   Topic9  -5.2622  -0.0224
118      code   6102.121645   97372.633672   Topic9  -5.2570  -0.1499
9         get   5491.918915  103740.354172   Topic9  -5.3624  -0.3186

[780 rows x 6 columns], token_table=       Topic      Freq          Term
term                                
22542      4  0.993329     __dirname
7930       4  0.994534         _post
7930       7  0.002061         _post
16967      4  0.994435  access_token
16967      6  0.001326  access_token
...      ...       ...           ...
2332       8  0.001445          yyyy
2332       9  0.000723          yyyy
12983      5  0.992574       yyyy_mm
2657       8  0.997126    zygoteinit
2657       9  0.000961    zygoteinit

[3516 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 5, 4, 3, 8, 6, 2, 9])